In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
IMAGE_SIZE = [224, 224]

In [ ]:
model = Sequential()

Now just remember the architecture in mind and start adding the layers into the network. our Kernel_size is (3,3) and the pool_size is (2,2) always.

In [ ]:
    model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),name='vgg16'))
    model.add(Flatten(name='flatten'))
    model.add(Dense(256, activation='relu', name='fc1'))
    model.add(Dense(128, activation='relu', name='fc2'))
    model.add(Dense(1, activation='sigmoid', name='output'))


In [ ]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
vgg.summary()

58892288/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
______________________________________________________________

To use a pre-trained model just makes the middle layers non trainable and remove the last layer.

We are also going to do the same let’s makes the middle layers freeze.

In [ ]:
for layer in vgg.layers:
  layer.trainable = False

We have already removed the output layer by include_top = False.

Let’s add our own output layer with only one node.

In [ ]:
x = Flatten()(vgg.output)
prediction = Dense(1, activation='sigmoid')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

Now we need to compile our model so that we can train it. our problem is a binary classification problem so we are going to use binary_crossentropy and optimizer we are going to use as Adam and for metrics, we are going to use accuracy.

In [ ]:
model.compile(
  loss='categorical_crossentropy',
  #optimizer='adam',
  metrics=['accuracy']
)

Let’s generate training and validation data using the data generator

In [ ]:
train_path="/content/gdrive/My Drive/datasets/Buildings224/train"
test_path="/content/gdrive/My Drive/datasets/Buildings224/test"

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical')



Found 2415 images belonging to 5 classes.
Found 250 images belonging to 5 classes.


Train the model

In [ ]:
r = model.fit_generator(
  training_set,  
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
151/151 [==============================] - 1345s 9s/step - loss: 0.7416 - accuracy: 0.7170 - val_loss: 0.4341 - val_accuracy: 0.7664
Epoch 2/10
151/151 [==============================] - 1332s 9s/step - loss: 0.4044 - accuracy: 0.7251 - val_loss: 0.8083 - val_accuracy: 0.7584
Epoch 3/10
151/151 [==============================] - 1331s 9s/step - loss: 0.3310 - accuracy: 0.6921 - val_loss: 0.3722 - val_accuracy: 0.7664
Epoch 4/10
151/151 [==============================] - 1334s 9s/step - loss: 0.2860 - accuracy: 0.7191 - val_loss: 0.2676 - val_accuracy: 0.8176
Epoch 5/10
151/151 [==============================] - 1331s 9s/step - loss: 0.2342 - accuracy: 0.7578 - val_loss: 0.3197 - val_accuracy: 0.7728
Epoch 6/10
151/151 [==============================] - 1330s 9s/step - loss: 0.2040 - accuracy: 0.6605 - val_loss: 0.2821 - val_accuracy: 0.7264
Epoch 7/10
151/151 [==============================] - 1330s 9s/step - loss: 0.1657 - accuracy: 0.6872 - val_loss: 0.2227 - val_accuracy:

Test the model

In [ ]:
model.evaluate_generator(test_set)